In [1]:
from scripts import dataset, tools, iterator
from models import alexnet, regressionnet
from tensorflow.contrib.data import Iterator
from copy import copy
from tqdm import tqdm
import math, os
import tensorflow as tf
import numpy as np
tools.etc.set_GPU("0")

In [2]:
net, loss_op, pose_loss_op, train_op = regressionnet.create_regression_net(data_shape=(227,227,3),optimizer_type='adadelta',num_joints=14)
    
with net.graph.as_default():
    saver = tf.train.Saver()

    train_it = dataset.met("/var/data/MET2/activity-met_n10_ub_train.csv", Fliplr=True, Shuffle=True)
    test_it = dataset.met("/var/data/MET2/activity-met_n10_ub_test.csv", Fliplr=True, Shuffle=True)

    summary_writer = tf.summary.FileWriter("./out/", net.sess.graph)
    summary_op = tf.summary.merge_all()

[Mirroring Images & Coordinates]: 100%|██████████| 53/53 [00:00<00:00, 1487.90it/s]


In [4]:
iter_num = 50000
snapshot_step = 2000
global_step = None
cur_train_op = train_op

In [ ]:
with tf.device("/gpu:0"):
    with tqdm(total = iter_num) as pbar:
        for step in range(iter_num):
                
            tr_cost = 0.
            tr_cnt = 0
            for n in range(2):
                global_step, summary_str, _, loss_value = net.sess.run(
                    [net.global_iter_counter,summary_op,cur_train_op,pose_loss_op],
                    feed_dict={net.x : train_it.img_set[:476] if n==0 else train_it.img_set[476:],
                               'PoseInput/joints_ground_truth:0' : train_it.coor_set.reshape(952,-1)[:476] if n == 0 else train_it.coor_set.reshape(952,-1)[476:],
                               'PoseInput/joints_is_valid:0': train_it.joint_is_valid[:476] if n==0 else train_it.joint_is_valid[476:],
                               'lr/conv_lr:0': 1.,
                               'lr/fc_lr:0': 1.})
                tr_cost+=loss_value
                tr_cnt+=1
            
            global_step, summary_str, _, loss_value = net.sess.run(
                [net.global_iter_counter,summary_op,cur_train_op,pose_loss_op],
                feed_dict={net.x : test_it.img_set,
                           'PoseInput/joints_ground_truth:0' : test_it.coor_set.reshape(len(test_it.coor_set),-1),
                           'PoseInput/joints_is_valid:0': test_it.joint_is_valid,
                           'lr/conv_lr:0': 1.,
                           'lr/fc_lr:0': 1.})
            
            if step%snapshot_step==0 and step !=0:
                saver.save(net.sess, "./out/alexnet_"+str(step)+".ckpt")
            
            pbar.update(1)
            pbar.set_description("[ Step : "+str(step+1)+"]")
            pbar.set_postfix_str(" Train Loss : "+str(tr_cost/tr_cnt)+" Test Loss : "+str(loss_value))

[ Step : 5819]:  12%|█▏        | 5819/50000 [2:55:54<22:11:28,  1.81s/it,  Train Loss : 330.593109131 Test Loss : 259.05]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[ Step : 10375]:  21%|██        | 10375/50000 [5:14:58<19:26:54,  1.77s/it,  Train Loss : 125.840366364 Test Loss : 115.107]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[ Step : 12740]:  25%|██▌       | 12740/50000 [6:27:56<19:37:35,  1.90s/it,  Train Loss : 118.325061798